In [3]:
# ================================
# Colab Chatbot
# ================================

!pip -q install --upgrade openai ipywidgets
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, json
from openai import OpenAI
import ipywidgets as W
from IPython.display import display, clear_output

# ---- API key ----
KEY_PATH = "/content/drive/MyDrive/key.txt"
api_key = open(KEY_PATH).read().strip() if os.path.exists(KEY_PATH) else os.getenv("OPENAI_API_KEY")
assert api_key, "No API key found. Put it in /content/drive/MyDrive/key.txt or set OPENAI_API_KEY."
client = OpenAI(api_key=api_key)
MODEL = "gpt-4o-mini"

# ---- Persistence ----
SAVE_DIR = "/content/drive/MyDrive/simple_chatbot"
os.makedirs(SAVE_DIR, exist_ok=True)
LOG_PATH = os.path.join(SAVE_DIR, "chat_history.json")

SYSTEM_PROMPT = (
    "You are a concise, friendly assistant. Be clear and helpful. "
    "When code is requested, prefer short, runnable examples."
)

def load_history():
    if os.path.exists(LOG_PATH):
        try:
            with open(LOG_PATH, "r", encoding="utf-8") as f:
                data = json.load(f)
                if isinstance(data, list):
                    return data
        except Exception:
            pass
    return [{"role": "system", "content": SYSTEM_PROMPT}]

def save_history(msgs):
    with open(LOG_PATH, "w", encoding="utf-8") as f:
        json.dump(msgs, f, ensure_ascii=False, indent=2)

messages = load_history()

def chat_once(user_text, temperature=0.2, max_tokens=400):
    global messages
    messages.append({"role": "user", "content": user_text})
    resp = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    reply = resp.choices[0].message.content.strip()
    messages.append({"role":"assistant","content":reply})
    save_history(messages)
    return reply

# ---- UI widgets ----
out = W.Output(layout=W.Layout(border='1px solid #ccc', height='360px', overflow='auto'))
inp = W.Text(placeholder="Say hi or ask for code…", layout=W.Layout(flex='1 1 auto'))
send = W.Button(description="Send", button_style='primary')
reset = W.Button(description="Reset chat")
temp = W.FloatSlider(description="Temperature", value=0.2, min=0.0, max=1.0, step=0.05, readout=True)
maxtok = W.IntSlider(description="Max tokens", value=400, min=64, max=2048, step=32, readout=True)

def render_history():
    with out:
        clear_output(wait=True)
        for m in messages:
            if m["role"] == "system":
                continue
            who = "You" if m["role"] == "user" else "Bot"
            print(f"{who}: {m['content']}\n")

def on_send(_):
    text = inp.value.strip()
    if not text:
        return
    inp.value = ""
    # optimistically show user message
    with out:
        print(f"You: {text}\n")
    reply = chat_once(text, temperature=temp.value, max_tokens=maxtok.value)
    with out:
        print(f"Bot: {reply}\n")

def on_reset(_):
    global messages
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    save_history(messages)
    render_history()

send.on_click(on_send)
inp.on_submit(on_send)
reset.on_click(on_reset)

controls = W.HBox([inp, send, reset])
display(W.VBox([out, controls, temp, maxtok]))

# initial render
render_history()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.4 MB/s eta 0:00:00
Mounted at /content/drive
